In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import joblib 


In [2]:
df = pd.read_csv('card_transdata.csv')

In [3]:
df.head()

distance_from_home  distance_from_last_transaction  \
0           57.877857                        0.311140   
1           10.829943                        0.175592   
2            5.091079                        0.805153   
3            2.247564                        5.600044   
4           44.190936                        0.566486   

   ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                        1.945940              1.0        1.0   
1                        1.294219              1.0        0.0   
2                        0.427715              1.0        0.0   
3                        0.362663              1.0        1.0   
4                        2.222767              1.0        1.0   

   used_pin_number  online_order  fraud  
0              0.0           0.0    0.0  
1              0.0           0.0    0.0  
2              0.0           1.0    0.0  
3              0.0           1.0    0.0  
4              0.0           1.0    0.0

In [4]:
df.shape

(1000000, 8)

In [5]:
df.describe()

distance_from_home  distance_from_last_transaction  \
count      1000000.000000                  1000000.000000   
mean            26.628792                        5.036519   
std             65.390784                       25.843093   
min              0.004874                        0.000118   
25%              3.878008                        0.296671   
50%              9.967760                        0.998650   
75%             25.743985                        3.355748   
max          10632.723672                    11851.104565   

       ratio_to_median_purchase_price  repeat_retailer       used_chip  \
count                  1000000.000000   1000000.000000  1000000.000000   
mean                         1.824182         0.881536        0.350399   
std                          2.799589         0.323157        0.477095   
min                          0.004399         0.000000        0.000000   
25%                          0.475673         1.000000        0.000000   
50%                          0.997717         1.000000        0.000000   
75%                          2.096370         1.000000        1.000000   
max                        267.802942         1.000000        1.000000   

       used_pin_number    online_order           fraud  
count   1000000.000000  1000000.000000  1000000.000000  
mean          0.100608        0.650552        0.087403  
std           0.300809        0.476796        0.282425  
min           0.000000        0.000000        0.000000  
25%           0.000000        0.000000        0.000000  
50%           0.000000        1.000000        0.000000  
75%           0.000000        1.000000        0.000000  
max           1.000000        1.000000        1.000000

In [6]:
df

distance_from_home  distance_from_last_transaction  \
0                57.877857                        0.311140   
1                10.829943                        0.175592   
2                 5.091079                        0.805153   
3                 2.247564                        5.600044   
4                44.190936                        0.566486   
...                    ...                             ...   
999995            2.207101                        0.112651   
999996           19.872726                        2.683904   
999997            2.914857                        1.472687   
999998            4.258729                        0.242023   
999999           58.108125                        0.318110   

        ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                             1.945940              1.0        1.0   
1                             1.294219              1.0        0.0   
2                             0.427715              1.0        0.0   
3                             0.362663              1.0        1.0   
4                             2.222767              1.0        1.0   
...                                ...              ...        ...   
999995                        1.626798              1.0        1.0   
999996                        2.778303              1.0        1.0   
999997                        0.218075              1.0        1.0   
999998                        0.475822              1.0        0.0   
999999                        0.386920              1.0        1.0   

        used_pin_number  online_order  fraud  
0                   0.0           0.0    0.0  
1                   0.0           0.0    0.0  
2                   0.0           1.0    0.0  
3                   0.0           1.0    0.0  
4                   0.0           1.0    0.0  
...                 ...           ...    ...  
999995              0.0           0.0    0.0  
999996              0.0           0.0    0.0  
999997              0.0           1.0    0.0  
999998              0.0           1.0    0.0  
999999              0.0           1.0    0.0  

[1000000 rows x 8 columns]

In [7]:
label_encoders = {}
for column in ['repeat_retailer', 'used_chip', 'used_pin_number', 'online_order']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [8]:
X = df[['distance_from_home', 'distance_from_last_transaction', 'ratio_to_median_purchase_price',
        'repeat_retailer', 'used_chip', 'used_pin_number', 'online_order']]
y = df['fraud']  

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.055338644427102755


In [13]:
joblib.dump(model, 'Card_Transdata.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

['label_encoders.pkl']

In [ ]:
import gradio as gr
import pandas as pd
import joblib

# Load the trained model and label encoders
model = joblib.load('Card_Transdata.pkl')
label_encoders = joblib.load('label_encoders.pkl')

def predict_transaction(distance_from_home, distance_from_last_transaction,
                        ratio_to_median_purchase_price, repeat_retailer,
                        used_chip, used_pin_number, online_order):
    # Prepare the input data
    input_data = pd.DataFrame({
        'distance_from_home': [float(distance_from_home)],
        'distance_from_last_transaction': [float(distance_from_last_transaction)],
        'ratio_to_median_purchase_price': [float(ratio_to_median_purchase_price)],
        'repeat_retailer': [label_encoders['repeat_retailer'].transform([repeat_retailer])[0]],
        'used_chip': [label_encoders['used_chip'].transform([used_chip])[0]],
        'used_pin_number': [label_encoders['used_pin_number'].transform([used_pin_number])[0]],
        'online_order': [label_encoders['online_order'].transform([online_order])[0]]
    })
    
    # Make prediction using the loaded model
    prediction = model.predict(input_data)
    return prediction[0]

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_transaction,
    inputs=[
        gr.Textbox(label='Distance from Home (in km)'),
        gr.Textbox(label='Distance from Last Transaction (in km)'),
        gr.Textbox(label='Ratio to Median Purchase Price'),
        gr.Textbox(label='Repeat Retailer (0 or 1)'),
        gr.Textbox(label='Used Chip (0 or 1)'),
        gr.Textbox(label='Used Pin Number (0 or 1)'),
        gr.Textbox(label='Online Order (0 or 1)')
    ],
    outputs=gr.Textbox(label="Predicted Transaction Value"),
    title="Transaction Value Prediction",
    description="Predict the transaction value based on its characteristics."
)

iface.launch()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.40.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/route_utils.py", line 285, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1923, in process_api
    result = await self.call_function(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1508, in call_function
    prediction = await anyio.to_t